In [2]:
import cenpy
import contextily as ctx 
import segregation
%matplotlib inline
import matplotlib as mpl

In [3]:
import xyzservices.providers as xyz
import matplotlib.pyplot as plt #to make sure there are no errors when plotting a graph
from contextily import add_basemap
import folium 

In [4]:
import geopandas as gpd 
from census import Census 
from us import states
from shapely.geometry import Polygon

In [5]:
import pandas as pd

**Census Api**

In [18]:
acs=cenpy.products.ACS(year="latest")  
acs.filter_tables('OWNER', by='description')


,description,columns
table_name,,
B25087,MORTGAGE STATUS AND SELECTED MONTHLY OWNER COSTS,"[B25087_001E, B25087_002E, B25087_003E, B25087..."
B25088,MEDIAN SELECTED MONTHLY OWNER COSTS (DOLLARS) ...,"[B25088_001E, B25088_002E, B25088_003E]"
B25089,AGGREGATE SELECTED MONTHLY OWNER COSTS (DOLLAR...,"[B25089_001E, B25089_002E, B25089_003E]"
B25091,MORTGAGE STATUS BY SELECTED MONTHLY OWNER COST...,"[B25091_001E, B25091_002E, B25091_003E, B25091..."
B25092,MEDIAN SELECTED MONTHLY OWNER COSTS AS A PERCE...,"[B25092_001E, B25092_002E, B25092_003E]"
B25093,AGE OF HOUSEHOLDER BY SELECTED MONTHLY OWNER C...,"[B25093_001E, B25093_002E, B25093_003E, B25093..."
B25094,SELECTED MONTHLY OWNER COSTS,"[B25094_001E, B25094_002E, B25094_003E, B25094..."
B25095,HOUSEHOLD INCOME BY SELECTED MONTHLY OWNER COS...,"[B25095_001E, B25095_002E, B25095_003E, B25095..."
B992522,ALLOCATION OF MORTGAGE STATUS AND SELECTED MON...,"[B992522_001E, B992522_002E, B992522_003E, B99..."


In [ ]:
c = Census('f5e7cf46eaa83ae8323e6051f74d9ab363d0b957')

In [ ]:
# Sources: https://api.census.gov/data/2019/acs/acs5/variables.html; https://pypi.org/project/census/
bxr_race = c.acs5.state_county_tract(fields = ('NAME', 'B02001_003E','B02001_001E' ),
                                      state_fips = states.TX.fips, 
                                      county_fips = "029",
                                      tract = "*",
                                      year = 2019)

In [ ]:
bxr_df = pd.DataFrame(bxr_race)

In [ ]:
print(bxr_df.head(2))
print('Shape: ', bxr_df.shape)

access shapefile of bexar county

In [ ]:
bxr_census_tract = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2019/TRACT/tl_2019_48_tract.zip")

print(bxr_census_tract)

In [ ]:
print(bxr_census_tract.head(2))

print('Shape: ', bxr_census_tract.shape)

# Check shapefile projection
print("\nThe shapefile projection is: {}".format(bxr_census_tract.crs))

In [ ]:
bxr_census_tract.plot('COUNTYFP', legend=True, figsize=(20,20))

performing dataframe operations

In [ ]:
# Combine state, county, and tract columns together to create a new string and assign to new column
bxr_df["GEOID"] = bxr_df["state"] + bxr_df["county"] + bxr_df["tract"]

In [ ]:
bxr_df.head(2)

In [ ]:
# Remove columns
bxr_df = bxr_df.drop(columns = ["state", "county", "tract"])

# Show updated dataframe
bxr_df.head(2)

In [ ]:
# Check column data types for census data
print("Column data types for census data:\n{}".format(bxr_df.dtypes))

# Check column data types for census shapefile
print("\nColumn data types for census shapefile:\n{}".format(bxr_census_tract.dtypes))

# Source: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html

In [ ]:
# Join the attributes of the dataframes together
# Source: https://geopandas.org/docs/user_guide/mergingdata.html
bxr_merge = bxr_census_tract.merge(bxr_df, on = "GEOID")

# Show result
print(bxr_merge.head(2))
print('Shape: ', bxr_merge.shape)

In [ ]:
# Create new dataframe from select columns
bxr_population_tract = bxr_merge[["STATEFP", "COUNTYFP", "TRACTCE", "GEOID", "geometry", "B02001_003E", "B02001_001E"]]

# Show dataframe
print(bxr_population_tract.head(2))
print('Shape: ', bxr_population_tract.shape)

In [ ]:
# Dissolve and group the census tracts within each county and aggregate all the values together
# Source: https://geopandas.org/docs/user_guide/aggregation_with_dissolve.html
bxr_population_county = bxr_population_tract.dissolve(by = 'COUNTYFP', aggfunc = 'sum')

# Show dataframe
print(bxr_population_county.head(2))
print('Shape: ', bxr_population_county.shape)

In [ ]:
# Get poverty rate and store values in new column
bxr_population_county["Population_Rate"] = (bxr_population_county["B02001_003E"] + bxr_population_county["B02001_001E"])/ bxr_population_county["B02001_001E"] * 100


# Show dataframe
bxr_population_county.head(2)  

In [ ]:
# Create subplots
fig, ax = plt.subplots(1, 1, figsize = (20, 10))

# Plot data
# Source: https://geopandas.readthedocs.io/en/latest/docs/user_guide/mapping.html
bxr_population_county.plot(column = "Population_Rate",
                       ax = ax,
                       cmap = "RdPu",
                       legend = True)

# Stylize plots
plt.style.use('bmh')

# Set title
ax.set_title('Population Rates (%) in Bexar County', fontdict = {'fontsize': '25', 'fontweight' : '3'})